In [1]:
%run scripts/setup.py

Beginning download...
Starting pariza/bbc-news-summary Download
Completed pariza/bbc-news-summary Download
Starting rmisra/news-category-dataset Download
Completed rmisra/news-category-dataset Download
Download completed!
json converted to DataFrame!
Articles downloaded!
Moving and renaming files...
Shutil task completed!
